# Use
This notebook will be used to tests the functions of the python files

In [47]:
import numpy as np
import warnings
%load_ext autoreload
%autoreload 2
from hungarian import preprocess, alternate, hungarian

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from warnings import warn

# `hungarian.py`
## `preprocess(C)`

In [49]:
n=4
C = np.random.random(size=(n,n))
n,U,V,row,x = preprocess(C)
print(C-U[:,np.newaxis]-V,"\n",x)

[[0.17457484 0.2464904  0.         0.        ]
 [0.         0.59083242 0.04612399 0.00229155]
 [0.91647983 0.         0.1646719  0.48183768]
 [0.54768548 0.         0.27790428 0.04387045]] 
 [[False False  True False]
 [ True False False False]
 [False  True False False]
 [False False False False]]


In [50]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,5,7],
              [1,6,6,9],
              [3,6,2,2]])
n,U,V,row,x = preprocess(C)
print(C-U[:,np.newaxis] - V, row, U, V,"\n", x)

[[0 0 1 2]
 [0 4 3 5]
 [0 3 5 8]
 [1 2 0 0]] [0 None 3 None] [7 2 1 2] [0 2 0 0] 
 [[ True False False False]
 [False False False False]
 [False False False False]
 [False False  True False]]


seem ok
- The zero entries match the True entries
- Internal assertions satisfied
- Same results as exemple 4.3
## `alternate(C,U,V,row,k)`

In [51]:
n=4
C = np.random.randint(10,size=(n,n))
n,U,V,row,x = preprocess(C)
k=n-1
if np.any(row==k):
     print("Invalid candidate k, try again")
else:
    sink,pred,SU,LV = alternate(C,U,V,row,k)
    print(row)
    print(sink,pred,SU,LV)
    print(C-U[:,np.newaxis]-V)

[1 2 0 None]
3 [3 None 1 1] [ True  True False  True] [ True False  True  True]
[[3 2 0 4]
 [0 3 0 0]
 [0 0 0 3]
 [0 1 3 4]]


In [52]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,6,7],
              [1,6,6,9],
              [3,6,2,2]])
n,U,V,row,x = preprocess(C)
k=1
sink,pred,SU,LV = alternate(C,U,V,row,k)
print(sink,pred)

1 [1 0 None None]


## `hungarian(C)`

In [53]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,6,7],
              [1,6,6,9],
              [3,6,2,2]])
row,x,phi,U,V = hungarian(C)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)


ValueError: too many values to unpack (expected 5)

In [ ]:
# random integer
n=10
C = np.random.randint(0,10,(n,n))
row,x,phi,U,V = hungarian(C)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)
[9 7 1 0 5 8 6 4 3 2] [3 2 9 8 7 4 6 1 5 0] [1 1 0 0 2 2 0 1 1 0] [ 1  0  2  1  0  0  0 -1  1  0]
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [ ]:
# random gaussian
n=4
C = np.random.random(size=(n,n))
row,x,phi,U,V = hungarian(C)
print(C)
print(C-U[:,np.newaxis]-V)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)
[[0.88750979 0.30436604 0.06572024 0.50733359]
 [0.97961864 0.26847654 0.79512909 0.81578427]
 [0.65527225 0.54489216 0.46235794 0.24825756]
 [0.4622001  0.94396178 0.08535918 0.47082073]]
[[0.44494863 0.2386458  0.         0.44161336]
 [0.33430117 0.         0.52665255 0.54730773]
 [0.03017377 0.2966346  0.21410039 0.        ]
 [0.         0.8586026  0.         0.38546155]]
[3 1 0 2] [2 1 3 0] [0.06572024 0.26847654 0.24825756 0.08535918] [0.37684093 0.         0.         0.        ]
[[0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]]


In [ ]:
# random gaussian
n=10
d=10
a=np.random.randn(d,n)
b=np.random.randn(d,n)
C = np.sum((a[:,:,np.newaxis]-b[:,np.newaxis,:])**2,axis=0)
#print(C)
row,x,phi,U,V = hungarian(C)
#print(C-U[:,np.newaxis]-V)
#print(row,phi,U,V)
#print(1*x)

hungarian succed (feasibility and complementary slackness holds)


## Questions/remarks
- We must test the **scalar equality** $u_i+v_j=c_{ij}$ multiple times, for now done with a tolerance of proximity but it's not very absolute and strong. How resolve? Solutions:
    - Use integers by **rescaling**.(the more we rescale the slower it becomes)
    - During the dual updating, **save the index** where the constrain is becoming active (by saving the argmin, see how to do it)
    
    -> it is indeed true with tolerance but wont affect the convergence of the algorithm neither the final output significantly

- implementation of **sets** by **vectors**. Better by python sets?

    -> we'll see, not so bad 

- describe the functionment of algorithm? a sketch of background?

    -> yes, sketch the idea, describe the steps, invariants, stopping situation.

- with floats, there is sometimes a small transgression of the dual feasibility, has been dealed by adding a tolerance `tol=1e-5`.

# `transport.py`
## `gaussian_transport()`

In [ ]:
%load_ext autoreload
%autoreload 2
from transport import gaussian_transport, gaussian_discreatization
rng = np.random.default_rng(4321)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
d = 2
mean1, mean2 = rng.normal(size=d), rng.normal(size=d)
A1, A2 = rng.normal(size=(d,d)), rng.normal(size=(d,d))
cov1, cov2 = A1.T @ A1, A2.T @ A2
transport, W_true = gaussian_transport(mean1, cov1, mean2, cov2)

n=100
mu, nu, C = gaussian_discreatization(mean1, cov1, mean2, cov2, n, rng)
row,x,phi,U,V, W_hungarian = hungarian(C)

hungarian succed (feasibility and complementary slackness holds)


In [ ]:
print("W_true =",W_true)
#transported_mu = np.array([transport(mu[:,i] for i in range(n))]).T
#W_true_discretized = np.sum((mu[:,np.newaxis,:] - transported_mu)**2, axis=2) / n
print("W_hungarian =",W_hungarian)
print("Error =",W_hungarian-W_true)
print("cond(cov1)=",np.linalg.cond(cov1),"cond(cov12)=",np.linalg.cond(cov2))

W_true = 9.516457823506894
W_hungarian = 9.80885524433506
Error = 0.29239742082816633
cond(cov1)= 9.3826297462511 cond(cov12)= 2.474799938877613
